<a href="https://colab.research.google.com/github/FoleyLab/wptherml/blob/master/PV_Opt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This notebook demonstrates the global optimization of a 2-layer AR coating for a 2 $\mu m$ silicon
solar cell and should reproduce the efficiency reported in Table III ($L = 2$) in "Accelerating the discovery of multi-layer nanostructures with analytic differentiation of the transfer matrix equations" by J. F. Varner, D. Wert, A. Matari, R. Nofal, and J. J. Foley IV.

In [ ]:
!pip install wptherml==1.11b0

In [ ]:
from wptherml.wpml import multilayer
from wptherml.datalib import datalib
from matplotlib import pyplot as plt
from scipy.optimize import minimize
from scipy.optimize import basinhopping
import numpy as np
import time

### Define a 2 micron silicon slab with a SiO2 and AlN coating...
### we will optimize over the SiO2 and AlN coatings only.
### Si with 2-layer AR coating
dof2_structure = {
        'Material_List' : ['Air', 'AlN','SiO2', 'Si','Air'],
        ### Thicknesses just chosen arbitrarily, replace with "optimal" values
        'Thickness_List': [0, 10.14e-9, 35.70e-9, 2000e-9, 0],
        'Lambda_List': [300e-9, 4000e-9, 1000]
        }

pv_2l = multilayer(dof2_structure)
plt.plot(pv_2l.lambda_array*1e9, pv_2l.emissivity_array, 'blue', label='Si + 2L')
plt.xlim(300,900)
plt.ylabel('Absorptivity (%)')
plt.xlabel('Wavelength ($nm$)')
plt.legend()

In [ ]:
pv_2l.pv_conversion_efficiency()

print("  Conversion efficiency of bare Si+2L",pv_2l.conversion_efficiency_val*100)

In [ ]:
dim = 50
l1 = np.linspace(1e-9, 400e-9, dim)
l2 = np.linspace(1e-9, 400e-9, dim)

pv_eff = np.zeros((dim,dim))
ia = -1
for la in l1:
    pv_2l.d[1] = la
    ia+=1
    ib = -1
    for lb in l2:
        pv_2l.d[2] = lb
        ib+=1
        #print(pv_2l.d)
        pv_2l.fresnel()
        pv_2l.pv_conversion_efficiency()
        #print(la, lb, pv_2l.conversion_efficiency_val*100)
        pv_eff[ia,ib] = pv_2l.conversion_efficiency_val
        


In [ ]:
%store pv_eff

In [4]:
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers

In [1]:
%store -r pv_eff

In [2]:
print(pv_eff)

[[0.05746748 0.05851806 0.0602211  ... 0.06843384 0.06833128 0.06820273]
 [0.060024   0.06612314 0.07195844 ... 0.06265331 0.06239897 0.06219623]
 [0.06425934 0.07174694 0.07619161 ... 0.05460678 0.05444614 0.05440159]
 ...
 [0.06646511 0.06531474 0.06280235 ... 0.05212275 0.05222645 0.05231165]
 [0.06648317 0.06533992 0.06283316 ... 0.05212071 0.05215051 0.05218407]
 [0.06649679 0.06536524 0.06288096 ... 0.05191795 0.05197705 0.0520554 ]]


In [5]:
# Model / data parameters
num_classes = 10
input_shape = (28, 28, 1)

# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

# Scale images to the [0, 1] range
x_train = x_train.astype("float32") / 255
x_test = x_test.astype("float32") / 255
# Make sure images have shape (28, 28, 1)
x_train = np.expand_dims(x_train, -1)
x_test = np.expand_dims(x_test, -1)
print("x_train shape:", x_train.shape)
print(x_train.shape[0], "train samples")
print(x_test.shape[0], "test samples")


# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

11501568/11490434 [==============================] - 4s 0us/step
x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples


In [6]:
model = keras.Sequential(
    [
        keras.Input(shape=input_shape),
        layers.Conv2D(32, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Conv2D(64, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Flatten(),
        layers.Dropout(0.5),
        layers.Dense(num_classes, activation="softmax"),
    ]
)

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 26, 26, 32)        320       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 13, 13, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 11, 11, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 5, 5, 64)         0         
 2D)                                                             
                                                                 
 flatten (Flatten)           (None, 1600)              0         
                                                                 
 dropout (Dropout)           (None, 1600)              0

In [7]:
batch_size = 128
epochs = 15

model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_split=0.1)

Epoch 1/15
422/422 [==============================] - 10s 23ms/step - loss: 0.3649 - accuracy: 0.8893 - val_loss: 0.0828 - val_accuracy: 0.9783
Epoch 2/15
422/422 [==============================] - 10s 25ms/step - loss: 0.1125 - accuracy: 0.9659 - val_loss: 0.0557 - val_accuracy: 0.9833
Epoch 3/15
422/422 [==============================] - 12s 27ms/step - loss: 0.0829 - accuracy: 0.9752 - val_loss: 0.0510 - val_accuracy: 0.9873
Epoch 4/15
422/422 [==============================] - 11s 27ms/step - loss: 0.0704 - accuracy: 0.9783 - val_loss: 0.0424 - val_accuracy: 0.9883
Epoch 5/15
422/422 [==============================] - 12s 29ms/step - loss: 0.0620 - accuracy: 0.9812 - val_loss: 0.0378 - val_accuracy: 0.9893
Epoch 6/15
422/422 [==============================] - 13s 31ms/step - loss: 0.0557 - accuracy: 0.9825 - val_loss: 0.0362 - val_accuracy: 0.9893
Epoch 7/15
422/422 [==============================] - 13s 31ms/step - loss: 0.0508 - accuracy: 0.9842 - val_loss: 0.0363 - val_accuracy:

In [8]:
score = model.evaluate(x_test, y_test, verbose=0)
print("Test loss:", score[0])
print("Test accuracy:", score[1])

Test loss: 0.023866476491093636
Test accuracy: 0.9922000169754028


In [18]:
print(x_test[0,:])
print(y_test[0,:])

[[[0.        ]
  [0.        ]
  [0.        ]
  [0.        ]
  [0.        ]
  [0.        ]
  [0.        ]
  [0.        ]
  [0.        ]
  [0.        ]
  [0.        ]
  [0.        ]
  [0.        ]
  [0.        ]
  [0.        ]
  [0.        ]
  [0.        ]
  [0.        ]
  [0.        ]
  [0.        ]
  [0.        ]
  [0.        ]
  [0.        ]
  [0.        ]
  [0.        ]
  [0.        ]
  [0.        ]
  [0.        ]]

 [[0.        ]
  [0.        ]
  [0.        ]
  [0.        ]
  [0.        ]
  [0.        ]
  [0.        ]
  [0.        ]
  [0.        ]
  [0.        ]
  [0.        ]
  [0.        ]
  [0.        ]
  [0.        ]
  [0.        ]
  [0.        ]
  [0.        ]
  [0.        ]
  [0.        ]
  [0.        ]
  [0.        ]
  [0.        ]
  [0.        ]
  [0.        ]
  [0.        ]
  [0.        ]
  [0.        ]
  [0.        ]]

 [[0.        ]
  [0.        ]
  [0.        ]
  [0.        ]
  [0.        ]
  [0.        ]
  [0.        ]
  [0.        ]
  [0.        ]
  [0.        ]
  [0. 